In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

import numpy as np
import pandas as pd

In [2]:
tf.__version__

'2.10.1'

In [3]:
from tqdm.keras import TqdmCallback

c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test_ds = tf.data.Dataset.load(r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_test").cache()

train_ds = tf.data.Dataset.load(r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_train").cache()

train_ds_1 = train_ds.batch(len(train_ds))
test_ds_1 = test_ds.batch(len(test_ds))

portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_v2/portfolios").cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])

unique_item_ids = np.unique(np.concatenate(list(items_ids)))
unique_item_names = np.unique(np.concatenate(list(item_names)))
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# need these to initialize timestamp embedding layers in future steps

timestamps = np.concatenate(list(portfolios.map(lambda x: x["UNIX_TS"]).batch(100)))
max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

KeyboardInterrupt: 

In [ ]:
train_ds_dict = next(iter(train_ds_1))
test_ds_dict = next(iter(test_ds_1))
#test
data_dict = {
    'CDSACCNO': test_ds_dict['CDSACCNO'].numpy(),
    'STOCKCODE': test_ds_dict['STOCKCODE'].numpy(),
    'STOCKNAME': test_ds_dict['STOCKNAME'].numpy(),
    'GICS': test_ds_dict['GICS'].numpy(),
    'UNIX_TS': test_ds_dict['UNIX_TS'].numpy(),
    'RATING': test_ds_dict['RATING'].numpy(),
}

test_df = pd.DataFrame.from_dict(data_dict)

test_df = test_df.astype(
    {
        'CDSACCNO' : 'str',
        'STOCKCODE' : 'str',
        'STOCKNAME' : 'str',
        'GICS' : 'str'
        })

#train
data_dict = {
    'CDSACCNO': train_ds_dict['CDSACCNO'].numpy(),
    'STOCKCODE': train_ds_dict['STOCKCODE'].numpy(),
    'STOCKNAME': train_ds_dict['STOCKNAME'].numpy(),
    'GICS': train_ds_dict['GICS'].numpy(),
    'UNIX_TS': train_ds_dict['UNIX_TS'].numpy(),
    'RATING': train_ds_dict['RATING'].numpy(),
}

train_df = pd.DataFrame.from_dict(data_dict)

train_df = train_df.astype(
    {
        'CDSACCNO' : 'str',
        'STOCKCODE' : 'str',
        'STOCKNAME' : 'str',
        'GICS' : 'str'
        })

In [ ]:
len(portfolios)

157854

In [ ]:
from retrieval_recommender import Retriever

retriever = Retriever(
    use_timestamp = True,
    portfolios = portfolios
)

retriever.load_weights(r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_05_27\retriever_port_v2")

retriever.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


In [ ]:
from ranker_recommender import Ranker

ranker = Ranker(
    loss = tf.keras.losses.MeanSquaredError(),
    portfolios = portfolios
)

ranker.load_weights(r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_05_27\tf_listwise_ranking_2024_05_27_11_20")
ranker.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


In [ ]:
stock_info_loc = r"D:\dev work\recommender systems\Atrad_CARS\data\stock_data.xlsx"

In [ ]:
stock_info = pd.read_excel(stock_info_loc)
stock_info = stock_info.drop(['Unnamed: 0','buisnesssummary'],axis = 1)
stock_info = stock_info.rename(columns = {
    'symbol':'STOCKCODE',
    'name' : 'STOCKNAME',
    'gics_code' : 'GICS'
})
stock_info = stock_info[~stock_info['GICS'].isna()]

stock_info.shape
print("items data shape :: {}".format(stock_info.shape))

items_ds = tf.data.Dataset.from_tensor_slices(stock_info.to_dict(orient= 'list'))

items data shape :: (280, 3)


# Retriever function

In [ ]:
items_identiifiers = items_ds.map(lambda x: x["STOCKCODE"])
items_identiifiers = next(iter(items_identiifiers.batch(len(items_identiifiers))))
items_identiifiers.shape

TensorShape([280])

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(
    query_model = retriever.user_model,
    k = 10)
    
retriever_item_model = retriever.item_model
mapped_items = items_ds.batch(len(items_ds)).map(lambda x : retriever_item_model(x, map_ = True))

mapped_items_tensor = next(iter(mapped_items))
index.index(mapped_items_tensor, items_identiifiers)

In [ ]:
next(iter(test_ds))

{'STOCKCODE': <tf.Tensor: shape=(), dtype=string, numpy=b'OSEA'>,
 'CDSACCNO': <tf.Tensor: shape=(), dtype=string, numpy=b'HDF-40956-LI/00'>,
 'RATING': <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
 'STOCKNAME': <tf.Tensor: shape=(), dtype=string, numpy=b'OVERSEAS REALTY (CEYLON) PLC'>,
 'GICS': <tf.Tensor: shape=(), dtype=string, numpy=b'Real Estate'>,
 'UNIX_TS': <tf.Tensor: shape=(), dtype=float32, numpy=1663266600.0>}

In [ ]:
# a test user ID & timestamp examples

test_user = str('RPS-797423181-VN/00')
test_timestamp = 1664821800.0

In [ ]:
seen_items = train_df.loc[train_df.CDSACCNO == test_user].STOCKCODE.values.reshape(1, -1)
seen_items

array([['HSIG', 'CARS', 'LMF', 'ASIR', 'HNBF', 'TJL', 'TAFL', 'RICH',
        'RCL', 'LDEV', 'SIGV', 'CALT', 'AHUN', 'CIC', 'LFIN', 'DIPD',
        'SHOT', 'LIOC', 'JKH', 'BIL', 'CONN', 'SHL', 'CHOT', 'MAL',
        'COMB', 'SIRA', 'CITH', 'EDEN', 'AGST', 'HAYL', 'CTBL', 'RHTL',
        'JINS', 'SAMP', 'TILE', 'SEYB', 'SINS', 'HOPL', 'KHL', 'ACL',
        'MGT', 'DIST', 'FCT', 'PEG', 'EXPO', 'SCAP', 'CFVF', 'TRAN',
        'HNB', 'KFP', 'JKL']], dtype=object)

In [ ]:
# including previously seen queries

_, recommendations = index(
    (
        tf.constant([test_user]),
        tf.constant([test_timestamp])
        )
    )

recommendations = [reco.decode('utf-8') for reco in recommendations.numpy().flatten()]
print(f"Recommendations for user %s: {recommendations}" %(test_user))
# titles[0]

Recommendations for user RPS-797423181-VN/00: ['BBH', 'HSIG', 'MAL', 'JKL', 'SIGV', 'TAJ', 'LIOC', 'BALA', 'CARS', 'HOPL']


In [ ]:
# excluding previously seen items

_, recommendations = index.query_with_exclusions(
    queries = (
        tf.constant([test_user]),
        tf.constant([test_timestamp])
        ),
    exclusions = seen_items
    )

recommendations = [reco.decode('utf-8') for reco in recommendations.numpy().flatten()]
print(f"Recommendations for user %s: {recommendations}" %(test_user))
# titles[0]

Recommendations for user RPS-797423181-VN/00: ['BBH', 'TAJ', 'BALA', 'CABO', 'ABAN', 'DOCK', 'HAPU', 'STAF', 'MASK', 'CTLD']


# Ranker function

In [ ]:
stock_info.head(2)

,STOCKCODE,STOCKNAME,GICS
0,HBS,hSenid Business Solutions PLC,45103010 - Application Software
1,TYRE,KELANI TYRES PLC,Automobiles & Components


In [ ]:
code2name = dict(zip(stock_info.STOCKCODE, stock_info.STOCKNAME))
code2gics = dict(zip(stock_info.STOCKCODE, stock_info.GICS))

In [ ]:
names = np.array([code2name[code] for code in recommendations])
gics = np.array([code2gics[code] for code in recommendations])

In [ ]:
user = {
    'CDSACCNO' : np.array([test_user]),
    'STOCKCODE' : np.array(recommendations).reshape(-1,10),
    'GICS' : gics.reshape(-1,10),
    'STOCKNAME' : names.reshape(-1,10)
    }

In [ ]:
pred_ratings = ranker(user)
pred_ratings.numpy().flatten()

array([1.0077842, 1.3442278, 1.9428025, 0.9984654, 3.0367675, 2.330201 ,
       0.9909103, 1.8309041, 2.17702  , 1.4371008], dtype=float32)

In [ ]:
recommendations_w_ratings = pd.DataFrame()
recommendations_w_ratings['STOCKCODE'] = recommendations
recommendations_w_ratings['PRED_RATING'] = pred_ratings.numpy().flatten()
recommendations_w_ratings = recommendations_w_ratings.sort_values( by = ['PRED_RATING'], ascending= False)
recommendations_w_ratings

,STOCKCODE,PRED_RATING
4,ABAN,3.036767
5,DOCK,2.330201
8,MASK,2.177020
2,BALA,1.942803
7,STAF,1.830904
9,CTLD,1.437101
1,TAJ,1.344228
0,BBH,1.007784
3,CABO,0.998465
6,HAPU,0.990910


# complete recommender function

In [ ]:
def recommend_(CDSACCNO, timestamp = 1664821800.0):

    seen_items = train_df.loc[train_df.CDSACCNO == CDSACCNO].STOCKCODE.values.reshape(1, -1)

    
    _, recommendations = index.query_with_exclusions(
    queries = (
        tf.constant([CDSACCNO]),
        tf.constant([timestamp])
        ),
    exclusions = seen_items
    )

    recommendations = [reco.decode('utf-8') for reco in recommendations.numpy().flatten()]
    # print(f"Recommendations for user %s: {recommendations}" %(test_user))

    user = {
    'CDSACCNO' : np.array([CDSACCNO]),
    'STOCKCODE' : np.array(recommendations).reshape(-1,10),
    'GICS' : gics.reshape(-1,10),
    'STOCKNAME' : names.reshape(-1,10)
    }

    pred_ratings = ranker(user)

    recommendations_w_ratings = pd.DataFrame()
    recommendations_w_ratings['STOCKCODE'] = recommendations
    recommendations_w_ratings['PRED_RATING'] = pred_ratings.numpy().flatten()
    recommendations_w_ratings = recommendations_w_ratings.sort_values( by = ['PRED_RATING'], ascending= False).reset_index(drop = True)

    user_test_port_ = test_df.iloc[test_users_.groups[CDSACCNO]].sort_values('RATING', ascending = False)
    recommendations_w_ratings = recommendations_w_ratings.join(user_test_port_.set_index('STOCKCODE'), on = 'STOCKCODE')[['STOCKCODE','PRED_RATING','RATING']]
    
    return recommendations_w_ratings


In [ ]:
import random

test_user = random.choice(train_df.CDSACCNO.unique())
test_timestamp = 1664821800.0
print(test_user)
recommend_(test_user, test_timestamp)

HDF-811281298-VN/00


KeyError: 'HDF-811281298-VN/00'

In [ ]:
test_users_ = test_df.groupby('CDSACCNO')
train_users_ = train_df.groupby('CDSACCNO')

In [ ]:
name = 'SBL-762412608-VN/00'

In [ ]:
user_test_port_ = test_df.iloc[test_users_.groups[name]].sort_values('RATING', ascending = False)
user_test_port_

,CDSACCNO,STOCKCODE,STOCKNAME,GICS,UNIX_TS,RATING
2594,SBL-762412608-VN/00,LIOC,LANKA IOC PLC,Energy,1.692729e+09,4.0
4,SBL-762412608-VN/00,CINS,CEYLINCO INSURANCE PLC,Insurance,1.704911e+09,3.0
14747,SBL-762412608-VN/00,HHL,HEMAS HOLDINGS PLC,Capital Goods,1.686595e+09,3.0
15204,SBL-762412608-VN/00,TJL,TEEJAY LANKA PLC,Consumer Durables & Apparel,1.683830e+09,2.0


In [ ]:
user_train_port_ = train_df.iloc[train_users_.groups[name]].sort_values('RATING', ascending = False)
# user_train_port_

In [ ]:
recommendations_ = recommend_(name, test_timestamp)
recommendations_

,STOCKCODE,PRED_RATING
8,CWM,3.386832
0,NEST,2.693880
7,TAFL,2.606198
6,CTC,2.532045
2,LLUB,2.517207
5,UCAR,2.477804
9,CARE,2.232112
1,KGAL,1.959322
3,LWL,1.746278
4,LCEY,1.658396


In [ ]:
recommendations_.iloc[5]

STOCKCODE          UCAR
PRED_RATING    2.477804
Name: 5, dtype: object

In [ ]:
recommendations_.join(user_test_port_.set_index('STOCKCODE'), on = 'STOCKCODE')

,STOCKCODE,PRED_RATING,CDSACCNO,STOCKNAME,GICS,UNIX_TS,RATING
8,CWM,3.386832,NaN,NaN,NaN,NaN,NaN
0,NEST,2.693880,NaN,NaN,NaN,NaN,NaN
7,TAFL,2.606198,NaN,NaN,NaN,NaN,NaN
6,CTC,2.532045,NaN,NaN,NaN,NaN,NaN
2,LLUB,2.517207,NaN,NaN,NaN,NaN,NaN
5,UCAR,2.477804,NaN,NaN,NaN,NaN,NaN
9,CARE,2.232112,NaN,NaN,NaN,NaN,NaN
1,KGAL,1.959322,NaN,NaN,NaN,NaN,NaN
3,LWL,1.746278,NaN,NaN,NaN,NaN,NaN
4,LCEY,1.658396,NaN,NaN,NaN,NaN,NaN
